# Web scraping of the pytorch document

## Importing libraries

In [49]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.parse import urljoin
import json
# import pandas as pd

In [ ]:
base_url="https://pytorch.org/docs/stable/"
url="nn.html"

response=requests.get(urljoin(base_url,url))

html=response.text

In [15]:
with open("text.txt","w") as f:
    f.write(html)

In [16]:
soup=bs(html,'html.parser')

In [43]:
def extract_doc_info(soup, base_url="https://pytorch.org/docs/stable/"):
    result = {}

    try:
        # Extract title from <span class="pre">
        title = soup.find("span", class_="pre").get_text(strip=True)
        result["title"] = title
    except AttributeError:
        result["title"] = None

    try:
        # Extract relative link from <a>
        a_tag = soup.find("a", class_="reference internal")
        relative_link = a_tag.get("href")
        full_url = urljoin(base_url, relative_link)
    except Exception:
        result["page_text"] = ""
        result["page_links"] = []
        return result

    try:
        # Request the page
        response = requests.get(full_url)
        page_soup = bs(response.text, "html.parser")

        # Extract page text
        page_text = page_soup.find("div",class_="pytorch-content-left").get_text(separator="\n", strip=True)
        result["page_text"] = page_text

        # Extract all links
        links = [urljoin(full_url, a.get("href")) for a in page_soup.find("div",class_="pytorch-content-left").find_all("a", href=True)]
        result["page_links"] = links

    except Exception as e:
        result["page_text"] = ""
        result["page_links"] = []

    return result


In [50]:
documents=soup.find_all("tr")
title=soup.find("h1")
title_text=title.next
print(title_text)

heading=title.find_next_sibling("p").text
print(heading)


torch.nn
These are the basic building blocks for graphs:


In [ ]:
with open("doc_info.json", "w", encoding="utf-8") as f:
    # Write the opening of the JSON object. Giving it the heading
    f.write('{\n')
    f.write(f'"title": {json.dumps(title_text)},\n')
    f.write(f'"heading": {json.dumps(heading)},\n')
    f.write('"documents": [\n')

    documents = soup.find_all("tr")
    for i, row in enumerate(documents):
        d = extract_doc_info(row)
        json.dump(d, f, ensure_ascii=False, indent=2)

        if i < len(documents) - 1:
            f.write(',\n')
        else:
            f.write('\n')

    f.write(']\n}')